<a href="https://colab.research.google.com/github/alizul01/machine-learning-course/blob/main/praktikum_2_tasks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Library Area

In [2]:
import tensorflow as tf
import numpy as np
import os
import time

## Preprocessing

In [4]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


In [5]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [6]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [7]:
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


## Text Processing

In [8]:
example_texts = ['abcdefg', 'xyz']
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [9]:
ids_from_chars = tf.keras.layers.StringLookup(
vocabulary=list(vocab), mask_token=None)

In [10]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [11]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [12]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [13]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [14]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [15]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [16]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [17]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [18]:
seq_length = 100

In [19]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [20]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [21]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [22]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [23]:
dataset = sequences.map(split_input_target)

In [24]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [25]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [26]:
vocab_size = len(ids_from_chars.get_vocabulary())

embedding_dim = 256

rnn_units = 1024

In [27]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [28]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [29]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [30]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [31]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [32]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b" one attend him with a silver basin\nFull of rose-water and bestrew'd with flowers,\nAnother bear the "

Next Char Predictions:
 b"'He- Rqngbf3'jYCTkI- AFZgGrOM$mhNVVk- mXMxjlG&[UNK]?'jRmt[UNK]\n3Vh&HZapnd TVF,qXnkVhH$Ajn,\n!hJYNjfXr,$&hGSdX"


In [33]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [34]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.190919, shape=(), dtype=float32)


In [35]:
tf.exp(example_batch_mean_loss).numpy()

66.08349

In [37]:
model.compile(optimizer='adam', loss=loss)

In [38]:
checkpoint_dir = './training_checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [39]:
EPOCHS = 20

In [40]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
172/172 [==============================] - 17s 61ms/step - loss: 2.7234
Epoch 2/20
172/172 [==============================] - 13s 58ms/step - loss: 1.9979
Epoch 3/20
172/172 [==============================] - 13s 57ms/step - loss: 1.7188
Epoch 4/20
172/172 [==============================] - 12s 62ms/step - loss: 1.5559
Epoch 5/20
172/172 [==============================] - 12s 61ms/step - loss: 1.4567
Epoch 6/20
172/172 [==============================] - 13s 57ms/step - loss: 1.3886
Epoch 7/20
172/172 [==============================] - 11s 57ms/step - loss: 1.3360
Epoch 8/20
172/172 [==============================] - 12s 56ms/step - loss: 1.2915
Epoch 9/20
172/172 [==============================] - 11s 57ms/step - loss: 1.2513
Epoch 10/20
172/172 [==============================] - 12s 58ms/step - loss: 1.2126
Epoch 11/20
172/172 [==============================] - 12s 59ms/step - loss: 1.1730
Epoch 12/20
172/172 [==============================] - 11s 57ms/step - loss: 1.1328
E

In [41]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)

    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    predicted_logits = predicted_logits + self.prediction_mask

    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    predicted_chars = self.chars_from_ids(predicted_ids)
    return predicted_chars, states

In [42]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [43]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
If sorelious Capulet which your tragh murder
You stand fast; and then we buy with solemn reverent
In my horse tears have done to the phocembly
A gaggers arm our coverity: offer
They're short, as thou, advise: mistake,
As shall remain a pox our royal enter;
And therein will speak as deep as neat.

Provost:
This is his thim!

PROSPERO:
Do so, then my awnable together
Late o'erpower that he shall nevet ribam
An every fortune fence am not ones, give
me your horse serves, or valiant William wretched,
And yet not hot;'t.

Clown:
Hath a husband till he will be the firmame.
This will I met, we show me here bring me.
And say I know thy words so well as one,
Which without your mistress could to Wadower Juliet's daughter,
Now thou couldst pile flainny-tongue--gain with
shiftless brandings hy wife for an itle passage,
Here lives not need to devose you:
Deay heavy to myself and very weaphy.

LUCIO:
Even truth, for truth and till I left him.

KING HENRY VI:
Pardon her, my shapes that doth be 

In [44]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\nOn which is thy seory woes, like a sea-cack?\nFor forbid! Why, so I am asleep;\nBut Front was time, offer but whilst thou,\nIrents that thy beam to serve his friends.\nBut he not son? behold, I prith! Implayation\nIf not on the air, you know his wife, the other,\nIf you were gone and tidings for my thoughts:\nAimerle of heaven, and did upright\nThe next gentleman to look on it; for therety\nWar worn to lament that there, did Richard and\nYet yet knew I here poor and rope thy way;\nWe cannot the king mis-stating with him:\nHis hathers are too rustle. Of him arm,\nThy due by--pay we show 'em tamed,\nOr that the inconstant doth begat\nFormerly report in the hatches: I fear:\nI am an executioner, his grave.\n\nEDWARD:\nNo; I come return to live in pity\nAnd through the vantage of the flowers: my fandship.\nTreasons a thousand children but as they\nTo thrive I want not partly turns.\nHalf my inform, mislehs a duke to leave with thee!\nThou shouldst thy dignity, that I 

In [45]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [46]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
The boy is temper'd: she is so made again,
That he would buzzard the life-woman contented.
The trib


In [47]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
    inputs, labels = inputs
    with tf.GradientTape() as tape:
      predictions = self(inputs, training=True)
      loss = self.loss(labels, predictions)
    grads = tape.gradient(loss, model.trainable_variables)
    self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

    return {'loss': loss}

In [48]:
model = CustomTraining(
  vocab_size=len(ids_from_chars.get_vocabulary()),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units)

In [49]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
       loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [50]:
model.fit(dataset, epochs=1)

172/172 [==============================] - 16s 66ms/step - loss: 2.7235


In [51]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
  start = time.time()

  mean.reset_states()
  for (batch_n, (inp, target)) in enumerate(dataset):
    logs = model.train_step([inp, target])
    mean.update_state(logs['loss'])

    if batch_n % 50 == 0:
      template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
      print(template)

  # saving (checkpoint) the model every 5 epochs
  if (epoch + 1) % 5 == 0:
    model.save_weights(checkpoint_prefix.format(epoch=epoch))

  print()
  print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
  print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
  print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.1729
Epoch 1 Batch 50 Loss 2.0240
Epoch 1 Batch 100 Loss 1.8954
Epoch 1 Batch 150 Loss 1.8681

Epoch 1 Loss: 1.9960
Time taken for 1 epoch 13.61 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 1.7960
Epoch 2 Batch 50 Loss 1.7572
Epoch 2 Batch 100 Loss 1.6633
Epoch 2 Batch 150 Loss 1.6514

Epoch 2 Loss: 1.7149
Time taken for 1 epoch 11.12 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.6080
Epoch 3 Batch 50 Loss 1.5777
Epoch 3 Batch 100 Loss 1.6170
Epoch 3 Batch 150 Loss 1.5151

Epoch 3 Loss: 1.5511
Time taken for 1 epoch 11.14 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.4712
Epoch 4 Batch 50 Loss 1.4641
Epoch 4 Batch 100 Loss 1.4092
Epoch 4 Batch 150 Loss 1.4350

Epoch 4 Loss: 1.4506
Time taken for 1 epoch 11.12 sec
_____________________________________________________________________

Tentu saja, mari kita bedakan antara dua kelas yang ditentukan dalam segmen kode yang disediakan.

1. **CustomTraining Class:**

   - Tujuan: Kelas ini digunakan untuk melatih model bahasa.
   - Mewarisi dari: Tampaknya mewarisi dari kelas bernama `Model Saya`, yang tidak ditentukan dalam kode yang disediakan.
   - Metode Utama:
     - `train_step(self, inputs)`: Metode ini menentukan langkah pelatihan khusus untuk model. Ini menghitung gradien, menerapkannya, dan memperbarui bobot model.
   - Penggunaan: Digunakan untuk melatih model menggunakan loop pelatihan khusus.

2. **OneStep Class:**

   - Tujuan: Kelas ini digunakan untuk menghasilkan teks menggunakan model bahasa terlatih.
   - Mewarisi dari: Tidak mewarisi kelas tertentu, tetapi tampaknya merupakan subkelas dari `tf.keras.Model`.
   - Metode Utama:
     - `generate_one_step(self, inputs, States=None)`: Metode ini menghasilkan teks selangkah demi selangkah. Dibutuhkan urutan masukan dan menghasilkan karakter berikutnya berdasarkan prediksi model yang dilatih.
   - Penggunaan: Digunakan untuk menghasilkan teks dengan memberinya urutan awal dan memungkinkannya menghasilkan karakter berikutnya selangkah demi selangkah.

Kedua kelas ini memiliki tujuan berbeda dalam konteks bekerja dengan model bahasa. Kelas `CustomTraining` bertanggung jawab untuk melatih model, sedangkan kelas `OneStep` digunakan untuk menghasilkan teks menggunakan model yang dilatih.